### Run the code below if pyspark operations throw random errors

In [ ]:
import shutil
shutil.rmtree("C:/Users/m/AppData/Local/Temp", ignore_errors=True)

Importing libraries, starting spark session and taking a look at the data

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
try:
    existing_spark = SparkSession.getActiveSession()
    if existing_spark:
        existing_spark.stop()
        print("Previous SparkSession stopped.")
except Exception as e:
    print(f"Could not get and stop the active SparkSession: {e}")

spark = SparkSession.builder.appName("Finance_Model_Training") \
    .master("spark://host.docker.internal:7077") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.memory", "8g") \
    .config("spark.default.parallelism", "8") \
    .getOrCreate().getOrCreate()

Previous SparkSession stopped.


Let's read the data and split it into train and test to do performance comparison later

In [28]:
df = spark.read.csv("/content/Train_data.csv", header=True, inferSchema=True)
train_df, test_df = df.randomSplit(weights=[0.8,0.2], seed=42)

In [29]:
train_df.show()

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_hos

In [30]:
train_df.columns

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'class']

### To understand what features we are playing with, description of all the features is given below:

##### **Basic features which describe the connection without looking at the payload**
- **duration** – Length (in seconds) of the connection.
- **protocol_type** – Network protocol used (e.g., TCP, UDP, ICMP).
- **service** – Destination service (e.g., http, telnet, ftp).
- **flag** – Status flag of the connection (e.g., S0, SF, REJ).
- **src_bytes** – Number of data bytes sent from source to destination.
- **dst_bytes** – Number of data bytes sent from destination to source.
- **land** – 1 if connection is from/to the same host/port; 0 otherwise.
- **wrong_fragment** – Number of wrong fragments in the packet.
- **urgent** – Number of urgent packets.
##### **Content features which describe what is within the connection payload**
- **hot** – Number of “hot” indicators (e.g., suspicious commands).
- **num_failed_logins** – Number of failed login attempts.
- **logged_in** – 1 if successfully logged in; 0 otherwise.
- **num_compromised** – Number of compromised conditions.
- **root_shell** – 1 if root shell is obtained; 0 otherwise.
- **su_attempted** – 1 if “su root” command attempted; 0 otherwise.
- **num_root** – Number of “root” accesses.
- **num_file_creations** – Number of file creation operations.
- **num_shells** – Number of shell prompts opened.
- **num_access_files** – Number of attempts to access control files.
- **num_outbound_cmds** – Number of outbound commands.
- **is_host_login** – 1 if user is a “host login”; 0 otherwise.
- **is_guest_login** – 1 if user is a “guest login”; 0 otherwise.
##### **Traffic features based on the past 2 seconds wiindow for same host
- **count** – Number of connections to the same host in the past 2 seconds.
- **srv_count** – Number of connections to the same service.
- **serror_rate** – % of connections with “SYN” errors.
- **srv_serror_rate** – % of connections with “SYN” errors to the same service.
- **rerror_rate** – % of connections with “REJ” errors.
- **srv_rerror_rate** – % of REJ errors to the same service.
- **same_srv_rate** – % of connections to the same service.
- **diff_srv_rate** – % of connections to different services.
- **srv_diff_host_rate** – % of connections to the same service but different host.
- **dst_host_count** – Number of connections to the destination host.
- **dst_host_srv_count** – Connections to the destination host using the same service.
- **dst_host_same_srv_rate** – % of same-service connections among dst_host_count.
- **dst_host_diff_srv_rate** – % of different-service connections.
- **dst_host_same_src_port_rate** – % of connections from same source port.
- **dst_host_srv_diff_host_rate** – % of same-service connections to different hosts.
- **dst_host_serror_rate** – % of connections with SYN errors.
- **dst_host_srv_serror_rate** – % of same-service SYN errors.
- **dst_host_rerror_rate** – % of connections with REJ errors.
- **dst_host_srv_rerror_rate** – % of same-service REJ errors.
##### **Label**
- **class** - either "normal" or "anomally"

Let's see if the data has duplicates and/or missing values

In [31]:
if train_df.count() == train_df.na.drop(how="any").count():
    print("No missing values in the dataset")
else:
    print("Missing values, need to treat the dataset")

if train_df.count() == train_df.dropDuplicates().count():
    print("No duplicates in the dataset")
else:
    print("Duplicate values, need to treat the dataset")

No missing values in the dataset
No duplicates in the dataset


Lets check the data distribution for all the columns

In [32]:
train_df.describe().show()

+-------+------------------+-------------+-------+-----+------------------+------------------+--------------------+-------------------+------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------+-------------+--------------------+------------------+-----------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|summary|          duration|protocol_type|service| flag|         src_bytes|         dst_bytes|                land|     wrong_fragment|urgent|                hot|   num_failed_logins

You can notice that some of the columns min, max, average and median is the same value, hence they don't have any value variation. Dropping these columns will help make our training more efficient

In [33]:
train_df = train_df.drop("num_outbound_cmds", "is_host_login")

We will not be checking for outliers in the data as they are valuable for the potential anomally detection i.e. financial fraud. We will look into class imbalance and resolve it further

In [34]:
train_df.groupBy().pivot("class").count().show()

+-------+------+
|anomaly|normal|
+-------+------+
|   9420| 10757|
+-------+------+



The classes are in the ratio of 45:55 which is decently balanced, hence we don't need to do further balancing in such scenario. Lets check if the datatypes are assigned accurately or if they need to be changed

In [35]:
train_df.show()

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_host_diff_

In [36]:
train_df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- srv_count: integer (nullable = true)
 |-- serror_rate: double (nullable = true)
 |-- srv_

The columns look fine and the correct values seem to be assigned. Lets encode all the string columns to indexed columns and then perform one hot encoding, so its readable to our ML model

In [37]:
# Getting all the column names which are string
columnList = [item[0] for item in train_df.dtypes if item[1].startswith('string')]

In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

index_cols = [c + "_index" for c in columnList]
ohe_cols = [c + "_ohe" for c in columnList]

# Step 1: Index string columns to numeric
indexers = [StringIndexer(inputCol=c, outputCol=c + "_index") for c in columnList]

# Step 2: OneHot encode the indexed columns
ohe = OneHotEncoder(inputCols=index_cols, outputCols=ohe_cols)

# Step 3: Build pipeline
pipeline = Pipeline(stages=indexers + [ohe])

# Fit and transform
encoder_model = pipeline.fit(train_df)
train_encoded_df = encoder_model.transform(train_df)
test_encoded_df = encoder_model.transform(test_df)

# To show the change from original value, indexed value and one hot encoded value
train_encoded_df.select(columnList + index_cols + ohe_cols).show()

index_cols.remove("class_index")
train_encoded_df = train_encoded_df.drop(*(index_cols))
test_encoded_df = test_encoded_df.drop(*(index_cols))

+-------------+-------+----+-------+-------------------+-------------+----------+-----------+-----------------+--------------+--------------+---------+
|protocol_type|service|flag|  class|protocol_type_index|service_index|flag_index|class_index|protocol_type_ohe|   service_ohe|      flag_ohe|class_ohe|
+-------------+-------+----+-------+-------------------+-------------+----------+-----------+-----------------+--------------+--------------+---------+
|         icmp|  eco_i|  SF|anomaly|                2.0|          5.0|       0.0|        1.0|        (2,[],[])|(65,[5],[1.0])|(10,[0],[1.0])|(1,[],[])|
|         icmp|  eco_i|  SF|anomaly|                2.0|          5.0|       0.0|        1.0|        (2,[],[])|(65,[5],[1.0])|(10,[0],[1.0])|(1,[],[])|
|         icmp|  eco_i|  SF|anomaly|                2.0|          5.0|       0.0|        1.0|        (2,[],[])|(65,[5],[1.0])|(10,[0],[1.0])|(1,[],[])|
|         icmp|  eco_i|  SF|anomaly|                2.0|          5.0|       0.0|       

You may notice that the one hot encoded values when ".show()" is used don't look like arrays. However, they are arrays. To understand what it means, I will use an example given below:

(65,[44],[1.0]) => the array has 65 values and the index 44 has a non-zero value i.e. 1.0

Alright, we have done plenty of data preprocessing, we can move on to normalizing the data but before that, we have to use Vector Assembler to combine input columns into single vector column

In [43]:
# Getting the list of input columns
input_cols = train_encoded_df.columns
for i in columnList:
    input_cols.remove(i)
input_cols.remove("class_index")
input_cols.remove("class_ohe")

In [45]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
assembler = VectorAssembler(inputCols=input_cols, outputCol="features_vector")
train_assembled_df = assembler.transform(train_encoded_df)
test_assembled_df = assembler.transform(test_encoded_df)

vec_assembler = VectorAssembler(
    inputCols=["class_ohe"],  # or any other OHE columns
    outputCol="class_ohe_vec"
)
train_assembled_df = vec_assembler.transform(train_assembled_df)
test_assembled_df = vec_assembler.transform(test_assembled_df)

In [46]:
scaler = MinMaxScaler(inputCol="features_vector", outputCol="scaled_features")
scaler_model = scaler.fit(train_assembled_df)
train_scaled_df = scaler_model.transform(train_assembled_df)
test_scaled_df = scaler_model.transform(test_assembled_df)

In [47]:
train_scaled_df.show()

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+-----------+-----------------+--------------+--------------+---------+--------------------+-------------+--------------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerr

## Model Training

### Random forest + Filter Method

In [48]:
train = train_scaled_df.select(["scaled_features", "class_index"])
test = test_scaled_df.select(["scaled_features", "class_index"])

In [49]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=100, maxDepth=10, seed=42)
rf_model = rf.fit(train)

In [50]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9944167497507478
AUC :  0.9998684739912715
F1 score :  0.9944309323850854


The number of trees and depth may be too high, let's reduce it to make the model less heavy. We know we are not overfitting as the model performs well on unseen data (training data).

In [51]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=40, maxDepth=7, seed=42)
rf_model = rf.fit(train)

In [52]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9844466600199402
AUC :  0.9994746955152912
F1 score :  0.9846009263235828


Let's reduce it more and see the performance

In [53]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=30, maxDepth=5, seed=42)
rf_model = rf.fit(train)

In [54]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9750747756729811
AUC :  0.9979205298267412
F1 score :  0.9753318557648677


Depth = 5 seems more appropriate, as the performance didn't drop as much and the complexity of the model has significantly dropped. Lets check the feature importance and discard all the features which are not significantly contributing to the model

In [55]:
rf_model.featureImportances

SparseVector(113, {0: 0.0003, 1: 0.1092, 2: 0.0849, 4: 0.0013, 6: 0.0039, 7: 0.0001, 8: 0.0645, 9: 0.004, 10: 0.0, 12: 0.0001, 13: 0.0, 16: 0.0, 17: 0.0267, 18: 0.0101, 19: 0.0549, 20: 0.0171, 21: 0.0133, 22: 0.0079, 23: 0.0716, 24: 0.0291, 25: 0.0049, 26: 0.01, 27: 0.1292, 28: 0.0345, 29: 0.057, 30: 0.0178, 31: 0.0118, 32: 0.0273, 33: 0.0055, 34: 0.004, 35: 0.0209, 36: 0.0034, 37: 0.0025, 38: 0.0056, 39: 0.0064, 40: 0.0141, 41: 0.0001, 42: 0.0073, 43: 0.0079, 44: 0.0004, 45: 0.0129, 46: 0.0, 47: 0.0, 48: 0.0003, 49: 0.0, 50: 0.0, 51: 0.0, 52: 0.0, 56: 0.0003, 61: 0.0007, 91: 0.0001, 92: 0.0, 93: 0.0, 103: 0.0675, 104: 0.0436, 105: 0.0031, 106: 0.0014, 107: 0.0, 108: 0.0002, 110: 0.0})

In [56]:
threshold = 0.01  # Adjust as needed
low_importance_indices = [i for i, score in enumerate(rf_model.featureImportances) if score < threshold]

In [57]:
len(low_importance_indices)

91

91 out of 113 features in the vector don't provide much value, hence we can discard them.

In [58]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.sql.functions import udf

def drop_indices_from_vector(vector, drop_indices):
    if not isinstance(vector, SparseVector):
        vector = Vectors.sparse(len(vector), [(i, v) for i, v in enumerate(vector)])

    kept_items = [(i, v) for i, v in zip(vector.indices, vector.values) if i not in drop_indices]
    new_indices, new_values = zip(*kept_items) if kept_items else ([], [])
    return SparseVector(len(vector), new_indices, new_values)

drop_udf = udf(lambda vec: drop_indices_from_vector(vec, low_importance_indices))

In [59]:
train_reduced_df = train_scaled_df.withColumn("features_reduced", drop_udf("scaled_features"))
test_reduced_df = test_scaled_df.withColumn("features_reduced", drop_udf("scaled_features"))

Let's retrain the model with reduced features and compare the performance

In [60]:
train = train_reduced_df.select(["scaled_features", "class_index"])
test = test_reduced_df.select(["scaled_features", "class_index"])

In [61]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=30, maxDepth=5, seed=42)
rf_model = rf.fit(train)

In [62]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9750747756729811
AUC :  0.9979205298267412
F1 score :  0.9753318557648677


The performance looks good, lets save the model!

In [63]:
rf_model.write().overwrite().save("model")